In [1]:
'''
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""
'''

'\nimport os\nos.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152\nos.environ["CUDA_VISIBLE_DEVICES"] = ""\n'

In [2]:
import gym
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
#from keras.constraints import MaxNorm
import numpy as np
import tensorflow as tf
import gc
import pickle
from keras.models import load_model

Using TensorFlow backend.


In [3]:
#Checking use of GPUs
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())
sess= K.get_session()

['/gpu:0']


In [4]:
#Initializing environment
env = gym.make('Humanoid-v2')
So = env.reset()
A = env.action_space.sample()

In [5]:
#Creating the model for policy
global policy_model
policy_model = Sequential()
policy_model.add(Dense(100,input_shape = np.append(So, A).shape,kernel_initializer='random_uniform', activation = 'relu' ))
policy_model.add(Dense(30,kernel_initializer='random_uniform', activation = 'relu'))
policy_model.add(Dense(1,kernel_initializer='random_uniform', activation='linear' ))

#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
policy_model.compile(loss="mse", optimizer= "sgd")
print(policy_model.summary())
#policy_model = load_model('policy.h5')

sess.run(tf.global_variables_initializer())

#Intializing eligibility traces for policy improvement
global policy_eligibility_traces
policy_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in policy_model.trainable_weights]

#Setting learning hyperparameters 
global policy_alpha
policy_alpha = 0.1
global policy_lambda
policy_lambda =  0.9

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               39400     
_________________________________________________________________
dense_2 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 42,461
Trainable params: 42,461
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
#Creating model for values 
global value_model
value_model = Sequential()
value_model.add(Dense(100,input_shape = So.shape, kernel_initializer='random_uniform',activation = 'relu' ))
value_model.add(Dense(30, kernel_initializer='random_uniform',activation = 'relu'))
value_model.add(Dense(1,kernel_initializer='random_uniform', activation= 'sigmoid'))

#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
value_model.compile(loss="mse", optimizer= "sgd")
print(value_model.summary())
#value_model = load_model('value.h5')

sess.run(tf.global_variables_initializer())

#Intializing eligibility traces for value improvement
global value_eligibility_traces
value_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in value_model.trainable_weights]

#Setting learning hyperparameters 
global value_alpha
value_alpha = 0.1
global value_lambda
value_lambda = 0.9

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               37700     
_________________________________________________________________
dense_5 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 31        
Total params: 40,761
Trainable params: 40,761
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
#Defining other hyperparameters

global average_reward
average_reward = 5 #pickle.load(open('average_reward.ferch','rb'))
global etaa
etaa = 0.01

In [8]:
#Auxiliary functions for weights update

def set_weights(model, weights):  
    if(model=='value'):
        i=0
        for layer in value_model.layers:
            if 'dropout' not in layer.name:
                layer.set_weights([weights[i],weights[i+1]])
                i+=2
    elif(model=='policy'):
        i=0
        for layerz in policy_model.layers:
            if 'dropout' not in layerz.name:
                layerz.set_weights([weights[i],weights[i+1]])
                i+=2
    else:
        print('error')

def get_value_weights():
    weights =[]
    for w in value_model.layers:
        weights+= w.weights
    return weights

def get_policy_weights():
    weights =[]
    for w in policy_model.layers:
        weights+= w.weights
    return weights

In [9]:
#Reward Tracking
rewardz_received = []
delta_rewardz_received = []

In [10]:
#Policy Cheats
min_policy_value = 1
max_policy_value = 2
min_policy_list, max_policy_list = [] , []

In [11]:
#Update weights
def update_weights(previous_state, action , reward, state,  terminal, skip_count):
    
    #Compute rewards
    global average_reward,rewardz_received,delta_rewardz_received
    delta_rewards = reward - average_reward*skip_count + value_model.predict(np.array([state]))[0][0] - value_model.predict(np.array([previous_state]))[0][0]    
    rewardz_received.append(reward)
    delta_rewardz_received.append(delta_rewards)
    print(delta_rewards)
    if(terminal):
        delta_rewards = reward - average_reward + 0 - value_model.predict(np.array([previous_state]))[0][0]
    thresh = average_reward*5
    if(delta_rewards>=thresh):
        delta_rewards =thresh
    if(delta_rewards<=-thresh):
        delta_rewards=-thresh
    average_reward = average_reward + etaa*delta_rewards
    
    #Compute value updates (eligibility traces and weights)
    global value_eligibility_traces
    value_gradients = value_model.optimizer.get_gradients(value_model.output, value_model.trainable_weights)
    value_gradients = [tf.clip_by_norm(gradient, 10) for gradient in value_gradients]
    value_eligibility_traces_op = [tf.add(tf.multiply(value_eligibility_traces[i], tf.constant(value_lambda,dtype= tf.float32)),value_gradients[i] ) for i in range(len(value_eligibility_traces))] 
    value_eligibility_traces = [tf.convert_to_tensor(x) for x in sess.run(value_eligibility_traces_op,feed_dict={value_model.input:np.array([previous_state])})]       
    value_weights = get_value_weights()
    value_weights_op =  [tf.add(value_weights[i], tf.multiply(tf.constant(value_alpha*delta_rewards,dtype=tf.float32), value_eligibility_traces[i])) for i in range(len(value_weights))]
    set_weights('value', sess.run(value_weights_op))
    
    #Compute policy updates (eligibility traces and weights)
    global policy_eligibility_traces
    policy_gradients = policy_model.optimizer.get_gradients(policy_model.output, policy_model.trainable_weights)
    policy_gradients = [tf.clip_by_norm(gradient, 10) for gradient in policy_gradients]
    
    #Correction on policy predictions
    global min_policy_value,max_policy_value, min_policy_list, max_policy_list
    policy_value = policy_model.predict(np.array([np.append(previous_state, action)]))[0][0]
    if(policy_value < 0):
        min_policy_value = - policy_value
        policy_value = 1
    policy_value = policy_value + min_policy_value
    if(policy_value> max_policy_value):
        max_policy_value = policy_value + 1
    policy_value = policy_value/max_policy_value
    policy_value = 1/ (policy_value+1)
    min_policy_list.append(min_policy_value)
    max_policy_list.append(max_policy_value)

    policy_eligibility_traces_op = [tf.add(tf.multiply(tf.constant(policy_lambda,dtype=tf.float32), policy_eligibility_traces[i]), tf.multiply(tf.constant(policy_value, dtype= tf.float32), policy_gradients[i])) for i in range(len(policy_gradients))]
    policy_eligibility_traces = [tf.convert_to_tensor(x) for x in sess.run(policy_eligibility_traces_op, feed_dict={policy_model.input:np.array([np.append(previous_state,action)])})]
    policy_weights = get_policy_weights()
    policy_weights_op =  [tf.add(policy_weights[i], tf.multiply( policy_eligibility_traces[i],tf.constant(policy_alpha*delta_rewards,dtype=tf.float32))) for i in range(len(policy_weights))]
    set_weights('policy', sess.run(policy_weights_op))

In [12]:
#Creating model for remembering preferences over each state
number_of_preferences = 3
global preference_model
preference_model = Sequential()
preference_model.add(Dense(100,input_shape = So.shape, activation = 'relu' ))
preference_model.add(Dense(30, activation = 'relu'))
preference_model.add(Dense(A.shape[0]*number_of_preferences, activation= 'linear'))

adam = Adam(lr=0.3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
preference_model.compile(loss="mse", optimizer=adam)
print(preference_model.summary())
#preference_model = load_model('preference.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 100)               37700     
_________________________________________________________________
dense_8 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_9 (Dense)              (None, 51)                1581      
Total params: 42,311
Trainable params: 42,311
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#Selecting an action
def select_action(state):
    
    number_of_new_actions = 3
    
    predicted_preferences = preference_model.predict(np.array([state]))[0].reshape((number_of_preferences,A.shape[0]))
    
    #FIXING MATRIX INCONSISTENCIES
    low = env.action_space.low
    high = env.action_space.high
    
    for y in range(predicted_preferences.shape[0]):
        for z in range(predicted_preferences.shape[1]):
            if(predicted_preferences[y][z] < low[z]):
                predicted_preferences[y][z] = low[z]
            if(predicted_preferences[y][z] > high[z]):
                predicted_preferences[y][z] = high[z]
                
    new_actions = env.action_space.sample().reshape(1, A.shape[0])
    
    for _ in range(number_of_new_actions-1):
        new_actions= np.concatenate((new_actions,env.action_space.sample().reshape(1, A.shape[0])), axis=0 )
    
    all_actions = np.concatenate((predicted_preferences, new_actions), axis=0)
    #pickle.dump(all_actions, open('all_actions.jf','wb'))
    all_actionz = np.concatenate((np.tile(So, (all_actions.shape[0], 1)), all_actions),axis=1)
    policy_preferences = policy_model.predict(all_actionz)
    
    global min_policy_value, max_policy_value
    actions_list = []
    for i in range(all_actions.shape[0]):
        actions_list.append((all_actions[i], policy_preferences[i][0]))
    actions_list = sorted(actions_list, key = lambda x: x[1], reverse = True)
    #print("Actions ="+ str(actions_list))
    probabilities =np.array([[(x[1]+min_policy_value)/max_policy_value for x in actions_list]])[0]
    ############
    #probabilities = sess.run(tf.contrib.layers.softmax(tf.constant([[x[1]+min_policy_value for x in actions_list]])))[0]
    ############
    
    p = np.random.rand()*probabilities.sum()
    i=0
    for j in range(probabilities.shape[0]):
        p-=probabilities[j]
        if(p<0):
            i=j
    
    best_actions = np.array([x[0] for x in actions_list[0:number_of_preferences]]).flatten()
    preference_model.fit(np.array([state]), np.array([best_actions]), verbose=0)
    return actions_list[j][0]
        
    

In [ ]:
#Main loop
S = env.reset()
#action_count=0
episode_count=0
save= True
total_reward =0 
total_reward_list=[]

skip_count = 0
skip_reward=0
while(True):
    
    #renders environment 
    #env.render()
    
    
    #Selects action according to stochastic policy
    action = select_action(S)
    #action_count+=1
    
    #Takes action 
    S1, reward, done, info = env.step(action)
    
    skip_reward += reward
    skip_count += 1
    
    #Updates weights
    #if(np.random.randint(10)%5==0):
    if(True):
        update_weights(S , action, skip_reward, S1 , done, skip_count)
        skip_reward = 0
        skip_count =0
    
    S = S1
    total_reward +=reward
    
    if(done):
        
        #print('We are now at '+str(episode_count))
        gc.collect()
        print(str(episode_count)+' --total reward ='+ str(total_reward))
        total_reward_list.append(total_reward)
        total_reward=0
        
        #Resets episode
        S = env.reset()
        #action_count=0
        episode_count+=1
        
        
        
        if(episode_count%5==0):
            
            if(save):
                print('saving models')
                print('average reward '+ str(average_reward))
                
                preference_model.save('preference.h5')
                policy_model.save('policy.h5')
                value_model.save('value.h5')
                
                pickle.dump(rewardz_received, open('rewardz_received.ferch','wb'))
                pickle.dump(delta_rewardz_received, open('delta_rewardz_received.ferch','wb'))
                pickle.dump(average_reward, open('average_reward.ferch','wb'))
                pickle.dump(total_reward_list, open('total_rewards_hist','wb'))
                pickle.dump(min_policy_list, open('min_policy','wb'))
                pickle.dump(max_policy_list, open('max_policy','wb'))
                
                #Restarting keras session 
                K.clear_session()
                sess = K.get_session()
                preference_model = load_model('preference.h5')
                policy_model = load_model('policy.h5')
                value_model = load_model('value.h5')

        
        #Resets eligibility traces
        global value_eligibility_traces, policy_eligibility_traces
        policy_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in policy_model.trainable_weights]
        value_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in value_model.trainable_weights]
    

-0.211502541377
-0.216919265111
-0.255987098735
-0.025406689697
-0.02799477203
-0.117715917411
0.0745639118165
-0.514802520957
-0.17108434288
-0.163591190351
-0.0396827650101
-0.0675015994995
-0.0716935780902
-2.37115953823
-0.207401213674
-0.0621381098276
0 --total reward =75.4261715733
-0.0700411889679
-0.052810959332
-0.0538879427908
-0.0584175120904
-0.0890174617542
-0.087010973636
-0.0496709411096
-0.0974464197578
-0.0617130309189
-0.18080334185
-0.0334882011544
-1.19097162494
-0.521307676014
-0.641472776088
-0.205986660322
-0.128335572637
-0.297287569074
0.0357123790799
-0.108310143512
-0.00580622813493
0.00804031668805
-0.0957827525682
-0.18503864045
1 --total reward =109.409350121
-0.0236884730429
-0.0552566569402
-0.0534587154462
-0.0514097320585
-0.0246298979205
-0.0509838654717
-0.00504990376168
0.00708914200922
-0.0945401939448
-0.0186567672506
-0.0144485879697
-0.0610459724147
-0.0481764983253
-0.037404472254
-1.39239819344
-0.693800278927
2 --total reward =75.9499912009
-

0.107449561046
0.440746471913
0.434605070991
0.195284608841
0.338331049633
0.213353043676
0.312020338361
0.290311423695
0.186363249436
0.322087549985
0.221838586676
0.256433936682
0.222117048893
0.277323121161
-0.00542613151002
0.232506261444
0.17664082713
22 --total reward =136.482699294
0.0924384479932
0.160917649186
0.171573492951
0.159646608602
0.180405976347
0.145552204806
0.129517017484
0.097810460851
0.113052206547
-2.5302273638
-0.528176104125
-0.344324165659
0.155328979963
0.0727182556853
0.133563420254
-0.483551562247
0.107290122251
0.189196849647
0.11552769457
-0.0281281582079
-0.0563594176974
0.199820521552
0.190404583339
0.16449497629
0.217921024412
0.181418259486
0.181492372899
0.159037708516
0.199873061482
23 --total reward =137.552618601
0.115016227385
0.105592985343
0.104636419833
0.124683088476
0.126984310483
0.12158997542
0.125303959229
0.122686184823
0.117244149762
-2.58290424687
-0.332285741425
-0.626040460191
-0.256597217748
-0.11515771655
-0.172220722984
-0.27489

0.233291005107
0.242261466912
0.200425964289
0.216409000143
0.222093588616
-0.801818689751
0.154087925063
-0.461451522438
0.181568434055
0.165812126008
0.172875266155
0.149970422451
0.182078754783
-0.321075069715
0.293974224416
0.232988376663
0.208694695825
0.270921921338
-0.000251098478834
0.407212930148
0.400764875496
0.385235457919
41 --total reward =190.526967499
-0.260128069309
0.206413458466
0.212306832697
0.15916676691
0.164800296929
0.193040608101
0.210399998535
0.202750490422
0.193124069282
0.181403208923
-2.67892743207
-0.131279042015
0.712328935625
0.750389495198
0.757521044828
0.174309416759
0.544850743307
0.779996405778
0.517469389124
0.827654067793
0.863426855513
0.857556916519
0.812112552371
0.807400038227
0.832300937479
0.670835267138
42 --total reward =131.939241629
-0.325307783367
0.124170576023
0.130535550332
0.120006276833
0.119188495326
0.122273329781
0.110167206369
0.127831539416
0.102122298207
0.0993891545234
-0.260543592943
-0.0227994337576
-1.264948408
-0.21335

0.272351656189
0.275348160483
0.292648448432
0.25633574268
0.260317268106
0.278067025284
0.243646252291
0.27167756857
-3.65416838169
-0.544265875121
-0.0221849988187
0.182149936439
0.179660387494
0.16164514967
0.169221286818
0.196709003219
0.197293696083
0.126170170337
0.0594192351965
-0.0164698917397
0.133762291091
-1.50163440588
0.00399743551129
-0.160005564236
0.264033554716
0.352560017379
0.223180672854
0.376184395515
0.342537643681
0.365380183798
0.317983605528
0.346549933029
0.357719715085
0.0206551700033
0.222091346859
0.324205289767
0.298160187422
61 --total reward =177.570920901
-0.0480714998427
0.277315176972
0.283691265934
0.244757185069
0.243368604718
0.287831717068
0.171353245597
0.22529336648
0.244270793399
-4.97539648244
-1.12257055339
-0.0851189798697
-0.177977838197
-0.0381920203256
-0.078019669812
-0.203654875595
-0.0484751296444
-0.0323920204729
-0.0200710345451
-0.0352424071257
-1.09525755111
-0.621719918782
-0.156446333242
0.0109709404109
-0.160796149053
0.04185207

0.124610535757
0.123414398233
0.108919752191
0.109726060686
-2.59470984186
-0.965628932212
-0.0820343161841
-0.291244327485
-0.370444317569
-0.242407412977
-0.193166934362
-0.190588813212
-0.228177283636
-0.151984110003
-0.180060006033
-0.174508870383
-0.198188287522
-0.189806226808
-0.200450820711
-0.162020270673
-0.459176704041
-0.361118560591
-0.233685600556
79 --total reward =121.538614591
saving models
average reward 4.70271810672
-0.20997480886
0.234900765362
0.228214886007
0.220535668852
0.217009199517
0.193201969879
0.179113047736
0.201369140742
0.231981632286
-1.45206196717
0.0607153924104
-0.271607201108
-0.417191203816
0.0252428979419
-0.544472969729
-0.217742271974
-0.231851986676
-0.17727079326
-0.20214648304
-0.210225955472
-0.211885142595
-0.25390293778
-0.250811881518
-0.729765726428
-0.519536227973
80 --total reward =113.745821921
-0.205248988289
0.22311438105
0.260845737109
0.231385020879
0.241094095596
0.253625965657
0.21376813768
0.242401501972
0.221481538655
0.1826

...

0.0866370625446
-0.103377688861
-0.0721297131954
-0.0175821405314
936 --total reward =137.854204059
0.371907617294
0.369460896877
0.33099326181
0.402592883916
0.363580818596
0.362855178976
0.34599696352
0.33135960709
0.38447492319
0.36933587144
-5.42296340865
-1.67493102904
-0.291438645651
-0.039048454926
0.282414624406
0.223153455489
0.307368156349
0.287058501164
0.23896988779
0.274201593325
0.25244821906
0.295785990487
0.294804140645
-0.454271908815
-0.499398937997
-0.12221303874
-0.0706957435676
-0.0052847670219
-0.0441157164018
-0.537536889109
-0.261061125594
-0.596411319708
-0.422402553705
-0.409735683042
-0.442721658533
937 --total reward =152.726326563
0.418935070656
0.417183024541
0.388705364143
0.417603023838
0.387311018366
0.391377893154
0.380806007436
0.41356135966
-2.6233831576
-0.819936856148
-1.15689215324
0.264469871588
0.276489927053
0.355489679135
0.183698774403
0.380429785948
0.354133003227
0.43232285757
0.391713529733
0.364347048256
0.360614386665
-0.113337728057
0.2

0.287527076045
0.409336103625
0.388838023212
0.413658821403
0.0641569903674
0.265158151321
0.257404063415
0.191360456887
0.0340543815865
0.127899618516
0.113869477841
0.381886698045
0.338125750173
0.339743245965
0.323448230014
0.345863174693
0.337675922262
0.30897573029
0.266884983412
-0.00312028630848
953 --total reward =160.369545251
0.242447083038
0.28097505069
0.275028332572
0.285334292523
0.276291025561
0.284406369575
0.277077692669
0.241305635357
0.272026771995
0.257613019681
-1.24575890309
0.212027583019
0.332903172542
0.234582047692
0.442739793381
0.441397984331
0.21708923361
-0.03983600893
0.421167375674
954 --total reward =92.2146904161
saving models
average reward 4.67709047381
0.193548653916
0.252749966511
0.267908227645
0.265160662484
0.242484927644
0.24132195477
0.232218467362
0.245459555249
0.193538055342
0.197728444823
-0.944193383299
-0.189963702349
-1.06302452092
-0.148373621716
0.00634324332155
0.0479943304943
-0.0188302346239
0.0464091787291
-0.191232083316
-0.00598

-0.248915331781
-0.184085340374
-0.182897487137
-0.328816837029
-0.17487757575
-0.427099641391
-0.263163049293
-0.233974508193
-0.811500531103
-0.206943653879
-0.126246777894
-0.169020994548
-0.183003168286
-0.720813790658
-0.475106562067
-0.232241205302
-0.0428005712144
969 --total reward =149.662640762
saving models
average reward 4.50451298549
0.401853370965
0.389127957273
0.373813910018
0.392700739404
0.403767046576
0.389015741702
0.363387035318
0.387673032945
-3.6202479558
-0.633070562004
0.0488187588584
-0.370020768153
0.21428445481
0.239310585995
0.262465926313
0.239585118972
0.239810052429
0.221916230941
0.157140631548
-0.0907063136163
0.0804127866661
0.0864559019024
0.301160169418
0.277578731663
0.297990015808
0.0725570354621
0.280962520579
0.0114530890293
-0.0218604340289
0.0711479060839
0.0398247475752
-0.043738451206
-0.0556288138462
0.075118656112
-0.0479394349912
0.020492805325
-0.0518663477812
-0.586334209451
-0.105118175936
-0.0546098202635
0.0298037488449
0.05676517164

0.0469141234041
0.027353978726
0.102208427478
0.0375717523439
0.0183382115253
-2.58305761769
-0.482553394502
-0.404949771817
-0.42004065083
-0.435343329619
-0.435180768634
-0.628064198439
-0.691975060295
985 --total reward =117.587083656
0.336549895622
0.377878969367
0.321451177633
0.358871766281
0.381004700535
0.328682452496
0.357048552244
0.321969223516
0.330428940103
0.319337888188
-0.982000247102
-0.44603929625
-0.387539654955
-0.202447968242
-0.172147375836
-0.306874861178
-0.337918990155
-0.334681146072
-0.326252899152
-0.480582320426
-0.465712559689
986 --total reward =94.7612592954
0.385650710322
0.372210006695
0.390543975528
0.396991665071
0.352288786515
0.335920992515
0.316216835714
0.338805360356
0.341740843802
-1.47810208551
-1.11753195933
-0.0535220662738
-0.122497698791
-0.0617903944779
-0.302142527644
0.00227352798144
0.0928170363458
-0.153130779594
0.184561268393
0.0190990287261
0.121758663573
0.133037436154
-1.46050387299
-0.254584198793
-0.543869596061
-0.703392095535

-0.03263774836
0.0484687995002
0.0454574567001
-0.317900914139
-0.209057714098
-0.0688033740514
-0.274887151923
-0.139922719326
-0.192806466321
-0.183060374102
-0.219641420889
-0.553029343177
-0.354005265207
-0.356425472473
-0.323789858288
-0.227712894575
-0.2555764668
-0.448125414028
-0.380822594754
-0.349142819434
-0.454896501405
-0.433916842767
-0.511436445799
-0.494530621245
-0.415363457921
-0.434431529196
-0.44007657886
-0.475007621796
-0.48019001998
-0.504529939636
-0.342638548129
-0.336865116521
-0.800540695462
-0.606994526673
-0.572723830223
1001 --total reward =221.528362607
0.20149632609
0.20110344484
0.205890072163
0.187215620082
0.210628873266
0.200910471713
0.196768026395
0.22462952361
0.168600981772
0.17926410442
-1.60668673333
-0.401387020577
0.00674942413015
-0.136380170052
-0.118176035844
-1.66544138565
-0.228714839697
-0.486517194524
-0.415064331152
-0.165132160567
-0.192242259896
-0.176719991608
-0.211652189435
-0.163742112163
1002 --total reward =108.675822835
0.258